# Practice Session 04: Basket analysis

Author: <font color="blue">Aniol Petit</font>

E-mail: <font color="blue">aniol.petit01@estudiant.upf.edu</font>

Date: <font color="blue">15/10/2024</font>

<font size="+2" color="blue">Additional results: experiments on cross-department association rules</font>

In [1]:
pip install apyori

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import csv
import gzip

from apyori import apriori

# 1. Playing with apyori

In [3]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]
                
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

<font size="+1" color="red">Replace this cell with your own example of transactions (at least 20 transactions). Execute the apriori algorithm, in which you should obtain at least <strong>two</strong> rules of the form ['A', 'B'] => ['C'], i.e., at least two rules having a 2-itemset in the antecedent and a 1-itemset in the consequent. Modify the transactions until you obtain such rules.</font>

In [4]:
transactions = [['Milk', 'Bread', 'Butter', 'Yogurt'],
    ['Milk', 'Eggs', 'Flour'],
    ['Bread', 'Butter', 'Eggs'],
    ['Bread', 'Eggs', 'Cheese'],
    ['Butter', 'Cheese', 'Yogurt'],
    ['Milk', 'Bread'],
    ['Milk', 'Butter', 'Eggs', 'Flour'],
    ['Bread', 'Eggs', 'Yogurt'],
    ['Eggs', 'Flour', 'Sugar'],
    ['Milk', 'Bread', 'Cheese'],
    ['Bread', 'Butter', 'Cheese', 'Eggs', 'Yogurt'],
    ['Milk', 'Yogurt', 'Cheese'],
    ['Butter', 'Flour'],
    ['Eggs', 'Cheese', 'Yogurt'],
    ['Milk', 'Butter', 'Sugar'],
    ['Bread', 'Yogurt', 'Butter'],
    ['Milk', 'Eggs', 'Sugar', 'Cheese'],
    ['Butter', 'Eggs'],
    ['Bread', 'Eggs', 'Yogurt', 'Butter'],
    ['Flour', 'Butter', 'Sugar']]



<font size="+1" color="red">Replace this cell with a markdown cell containing (1) a printout of the rules you have obtained, and (2) for each of those rules, indicate clearly how the support, confidence, and lift is calculated. Do not merely repeat the formula: indicate how each number is computed based on the transactions you provided, as if you were trying to verify that the numbers are correct.</font>

In [5]:
results = list(apriori(transactions, min_support=0.1, min_confidence=0.9, min_lift=1.0))
print_apyori_output(results)

Rules involving itemset ['Yogurt', 'Cheese', 'Butter']
['Cheese', 'Butter'] => ['Yogurt'] (support=0.1000, confidence=1.00, lift=2.50)

Rules involving itemset ['Eggs', 'Flour', 'Milk']
['Milk', 'Flour'] => ['Eggs'] (support=0.1000, confidence=1.00, lift=1.82)

Rules involving itemset ['Eggs', 'Yogurt', 'Butter', 'Bread']
['Eggs', 'Yogurt', 'Butter'] => ['Bread'] (support=0.1000, confidence=1.00, lift=2.22)



<b>Rule 1<b>: A = [Cheese, Butter, Yogurt]    B = [Cheese, Butter]    C = [Yogurt]

Support(A) = (Transactions containing A) / (Total Transactions) = 2/20 = 0.1

Confidence(B-->C) = (Transactions containing both B and C (A)) / (Transactions containing B) = 2/2 = 1

lift(B-->C) = (Confidence(B-->C)) / (Support(C)) = 1 / (Transactions containing C) / (Total Transactions) = 1 / (8/20) = 20/8 = 5/2 = 2.5



<b>Rule 2<b>: A = [Milk, Flour, Eggs]    B = [Milk, Flour]    C = [Eggs]

Support(A) = (Transactions containing A) / (Total Transactions) = 2/20 = 0.1

Confidence(B-->C) = (Transactions containing both B and C (A)) / (Transactions containing B) = 2/2 = 1

Lift(B-->C) = (Confidence(B-->C)) / (Support(C)) = 1 / (Transactions containing C) / (Total Transactions) = 1 / (11/20) = 20/11 = 1.82



<b>Rule 3<b>: A = [Yogurt, Bread, Butter, Eggs]    B = [Yogurt, Butter, Eggs]    C = [Bread]

Support(A) = (Transactions containing A) / (Total Transactions) = 2/20 = 0.1

Confidence(B-->C) = (Transactions containing both B and C (A)) / (Transactions containing B) = 2/2 = 1

Lift(B-->C) = (Confidence(B-->C)) / (Support(C)) = 1 / (Transactions containing C) / (Total Transactions) = 1 / (9/20) = 20/9 = 2.22

# 2. Load and prepare the shopping baskets

In [6]:
# LEAVE AS-IS

# File names
INPUT_PRODUCTS = "instacart-products.csv"
INPUT_TRANSACTIONS = "instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

In [7]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

<font size="+1" color="red">Replace this cell with your code for *select_from_departments*.</font>

In [8]:
def select_from_departments(products, products_ids, departments_ids):
    products_in_departments = []
    for id in products_ids:
        id = int(id)
        if products.loc[id].department_id in departments_ids:
            products_in_departments.append(id)
    return products_in_departments

<font size="+1" color="red">Replace this cell with code to test your function with three different test cases. Each test case is a list of items and a list of 1, 2, or 3 departments.</font>

In [9]:
def print_test_case(products, products_ids, departments_ids, n):
    print(f"Test case {n}:\n{products_ids}\n")

    print("Input products:")
    for product_id in products_ids:
        product_name = products.loc[product_id].product_name
        department_id = products.loc[product_id].department_id
        print(f"{product_id} {product_name} (dept {department_id})")

    selected_product_ids = select_from_departments(products, products_ids, departments_ids)
    
    print("\nSelected products:")
    for product_id in selected_product_ids:
        product_name = products.loc[product_id].product_name
        department_id = products.loc[product_id].department_id
        print(f"{product_id} {product_name} (dept {department_id})")


#Test case 1
products_ids = [150, 245, 812, 234, 49661, 6, 82, 47]
departments = [DEPT_ALCOHOL]
print_test_case(products, products_ids, departments, 1)
print("\n")
#Test case 2
products_ids = [6, 7, 52, 23, 91, 10, 11, 31, 653]
departments = [DEPT_DRINKS, DEPT_PHARMACY]
print_test_case(products, products_ids, departments, 2)
print("\n")
#Test case 3
products_ids = [14, 43243, 21, 54, 321, 49686, 432]
departments = [DEPT_BAKERY, DEPT_PETS, DEPT_CLEANING]
print_test_case(products, products_ids, departments, 3)


Test case 1:
[150, 245, 812, 234, 49661, 6, 82, 47]

Input products:
150 Brut Rosé (dept 5)
245 Poppy Seed Bagels (dept 3)
812 Healthy Grains Oats & Honey Clusters with Toasted Coconut Gluten Free (dept 14)
234 Tennessee Whiskey (dept 5)
49661 Porto (dept 5)
6 Dry Nose Oil (dept 11)
82 All Natural 100% Apple Juice (dept 7)
47 Onion Flavor Organic Roasted Seaweed Snack (dept 6)

Selected products:
150 Brut Rosé (dept 5)
234 Tennessee Whiskey (dept 5)
49661 Porto (dept 5)


Test case 2:
[6, 7, 52, 23, 91, 10, 11, 31, 653]

Input products:
6 Dry Nose Oil (dept 11)
7 Pure Coconut Water With Orange (dept 7)
52 Mirabelle Brut Rose (dept 5)
23 Organic Turkey Burgers (dept 12)
91 Kind Prenatal Once Daily (dept 11)
10 Sparkling Orange Juice & Prickly Pear Beverage (dept 7)
11 Peach Mango Juice (dept 7)
31 White Pearl Onions (dept 4)
653 Flax, Oat Bran & Whole Wheat Flour Tortillas (dept 3)

Selected products:
6 Dry Nose Oil (dept 11)
7 Pure Coconut Water With Orange (dept 7)
91 Kind Prenatal On

## 2.2. Read and filter transactions

<font size="+1" color="red">Replace this cell with your code to read transactions, keeping only items in DEPT_PHARMACY. Remember to stop after storing 5000 of the transactions read.</font>

In [10]:
def extract_transactions(departments, max_transactions=5000):
    transactions = []
    stored = 0
    read = 0
    # Open a compressed file
    with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:
        
        # Create a CSV reader
        reader = csv.reader(inputfile, delimiter=",")
        
        # Iterate through the CSV file
        for row in reader:
            # Convert to integers
            items = [int(x) for x in row]
            dept_items = select_from_departments(products, items, departments)
            if (len(dept_items)>0): #keep only transactions that are non-empty
                transactions.append(dept_items)
                stored+=1
            read+=1
            if read%1000 == 0:
                print(f"{read} transactions read and {stored} transactions stored")
            if stored >= 5000:
                break
    return transactions
                
departments1 = [DEPT_PHARMACY]         
transactions1 = extract_transactions(departments1)

1000 transactions read and 102 transactions stored
2000 transactions read and 219 transactions stored
3000 transactions read and 313 transactions stored
4000 transactions read and 422 transactions stored
5000 transactions read and 526 transactions stored
6000 transactions read and 619 transactions stored
7000 transactions read and 722 transactions stored
8000 transactions read and 810 transactions stored
9000 transactions read and 920 transactions stored
10000 transactions read and 1046 transactions stored
11000 transactions read and 1152 transactions stored
12000 transactions read and 1245 transactions stored
13000 transactions read and 1352 transactions stored
14000 transactions read and 1459 transactions stored
15000 transactions read and 1559 transactions stored
16000 transactions read and 1663 transactions stored
17000 transactions read and 1753 transactions stored
18000 transactions read and 1854 transactions stored
19000 transactions read and 1943 transactions stored
20000 trans

## 2.3. Extract association rules and comment on them

<font size="+1" color="red">Replace this cell with your code to extract association rules from the read transactions.</font>

In [11]:
results = list(apriori(transactions1, min_support=0.0005, min_confidence=0.7, min_lift=1.0))
print_apyori_output(results, products, 'product_name')

Rules involving itemset [23425, 5019]
['Nourish & Moisturize Shampoo'] => ['Nourish+ Moisturize Conditioner'] (support=0.0006, confidence=1.00, lift=1250.00)
['Nourish+ Moisturize Conditioner'] => ['Nourish & Moisturize Shampoo'] (support=0.0006, confidence=0.75, lift=1250.00)

Rules involving itemset [11007, 5663]
['Chocolate Energy Supplement'] => ['Chocolate Calming Supplement'] (support=0.0006, confidence=1.00, lift=1250.00)
['Chocolate Calming Supplement'] => ['Chocolate Energy Supplement'] (support=0.0006, confidence=0.75, lift=1250.00)

Rules involving itemset [42563, 10781]
['Coconut Milk Nourishing Shampoo'] => ['Coconut Milk Nourishing Conditioner'] (support=0.0006, confidence=0.75, lift=625.00)

Rules involving itemset [35224, 12246]
['Drink It Up Coconut Milk Natural Hawaiian Shampoo'] => ['Drink It Up Coconut Milk Conditioner'] (support=0.0006, confidence=0.75, lift=340.91)

Rules involving itemset [16122, 44138]
['Plant-Based Vanilla Chai Flavor Nutritional Shake Drink Mi

<font size="+1" color="red">Replace this cell with a brief commentary on what you would recommend to the shopping app considering the extracted association rules.</font>

I would recommend to place the shampoos and conditioners that are of the same brand one next to the other, since with a high confidence people that buy the shampoo of a determined brand they also buy the conditioner, and viceversa (for instance, The Nourish & Moisturize).

I would also say that it is interesting to place all chocolate supplements together, since people who buy one of any kind seem to be likely to buy also of some other kind.

## 2.4. Extract association rules and comment on them (other departments)

<font size="+1" color="red">Replace this cell with code to select a different set of departments (at least two, not DEPT_PHARMACY) and extract transactions again. Avoid replicating code when possible.</font>

In [12]:
departments2 = [DEPT_VEGGIES, DEPT_DRINKS]
transactions2 = extract_transactions(departments2)

1000 transactions read and 886 transactions stored
2000 transactions read and 1746 transactions stored
3000 transactions read and 2612 transactions stored
4000 transactions read and 3492 transactions stored
5000 transactions read and 4358 transactions stored


In [13]:
results = list(apriori(transactions2, min_support=0.001, min_confidence=0.8, min_lift=1.0))
print_apyori_output(results, products, 'product_name')

Rules involving itemset [8424, 24852, 16797]
['Broccoli Crown', 'Strawberries'] => ['Banana'] (support=0.0012, confidence=0.86, lift=4.93)

Rules involving itemset [9387, 24852, 45007]
['Granny Smith Apples', 'Organic Zucchini'] => ['Banana'] (support=0.0010, confidence=1.00, lift=5.75)

Rules involving itemset [13176, 19660, 27966]
['Spring Water', 'Organic Raspberries'] => ['Bag of Organic Bananas'] (support=0.0014, confidence=0.88, lift=6.29)

Rules involving itemset [13176, 26940, 27966]
['Organic Large Green Asparagus', 'Organic Raspberries'] => ['Bag of Organic Bananas'] (support=0.0014, confidence=0.88, lift=6.29)

Rules involving itemset [22935, 24964, 16759]
['Organic Garlic', 'Organic Carrot Bunch'] => ['Organic Yellow Onion'] (support=0.0010, confidence=1.00, lift=24.51)

Rules involving itemset [28985, 24852, 16797]
['Michigan Organic Kale', 'Strawberries'] => ['Banana'] (support=0.0018, confidence=0.90, lift=5.18)

Rules involving itemset [21137, 27966, 38159]
['Organic Ra

<font size="+1" color="red">Replace this cell with your commentary on the obtained rules.</font>

After trying some combinations of different departments, I have observed that there are not many relations involving different departments. However, in this case we can observe many interesting relations involving mainly the veggies department.

Many people are likely to buying bananas (or bags of organics bananas) when buying any other veggie product such as avocado, apples, raspberries, broccoli, etc. Furthermore, we can see a some important relations when buying other products. Organic yellow onion is frequently bought together with organic garlic and organic carrot, which makes sense since they are products that may come together in many recipees. It is interesting to see that the organic garlic is bought together with organic yellow onion (as seen) and with organic red radish, which differs from the organic carrot that appeared in the onion relation.

Finally, we can point out a strong relation between organic raspberries, organic yellow peaches and organic strawberries, where the latter is frequently bought together with the other two products.

<font size="+1" color="red">EXTRA POINTS</font>

In [14]:
def print_apyori_output_diff_dept(association_results, info=False, info_key=False, dept_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                # If info_key is provided, map antecedent and consequent to product information
                if info_key:
                    antecedent_info = [info.loc[x][info_key] for x in antecedent]
                    consequent_info = [info.loc[x][info_key] for x in consequent]
                else:
                    antecedent_info = antecedent
                    consequent_info = consequent
                
                # If dept_key is provided, compare departments
                if dept_key:
                    antecedent_depts = [info.loc[x][dept_key] for x in antecedent]
                    consequent_depts = [info.loc[x][dept_key] for x in consequent]
                    can_print = False
                    for dept in consequent_depts:
                        if dept not in antecedent_depts:
                            can_print = True
                            
                    if can_print==True:
                        confidence = rules.confidence
                        lift = rules.lift
                        print("Rules involving itemset %s" % itemset)
                        # Print the rule details
                        print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                              (antecedent_info, consequent_info, support, confidence, lift))
                        print()


In [15]:
departments3 = [DEPT_BAKERY, DEPT_VEGGIES, DEPT_BABIES, DEPT_DRINKS]
transactions3 = extract_transactions(departments3)

1000 transactions read and 909 transactions stored
2000 transactions read and 1793 transactions stored
3000 transactions read and 2684 transactions stored
4000 transactions read and 3589 transactions stored
5000 transactions read and 4481 transactions stored


In [16]:
results = list(apriori(transactions3, min_support=0.001, min_confidence=0.5, min_lift=1.0))
print_apyori_output_diff_dept(results, products, 'product_name', 'department_id')

Rules involving itemset [24852, 9421]
['Organic Raw Multigreen Kobmbucha'] => ['Banana'] (support=0.0012, confidence=0.55, lift=3.25)

Rules involving itemset [13176, 44310]
['Soft Pretzel Mini Buns'] => ['Bag of Organic Bananas'] (support=0.0010, confidence=0.50, lift=3.67)

Rules involving itemset [13176, 49215]
['Kids Sensible Foods Broccoli Littles'] => ['Bag of Organic Bananas'] (support=0.0018, confidence=0.50, lift=3.67)

Rules involving itemset [29553, 24852]
['Super Soft Taco Flour Tortillas'] => ['Banana'] (support=0.0012, confidence=0.67, lift=3.97)

Rules involving itemset [24852, 36606]
['Sprouted Multi-Grain Bread'] => ['Banana'] (support=0.0010, confidence=0.50, lift=2.98)

Rules involving itemset [38192, 24852]
['Pear & Mango Stage 2 Baby Food'] => ['Banana'] (support=0.0010, confidence=0.56, lift=3.31)

Rules involving itemset [45948, 24852]
['Gluten Free 7 Grain Bread'] => ['Banana'] (support=0.0018, confidence=0.53, lift=3.16)



It seems that again bananas and bags of organic bananas are a main character. In this case we see a strong relation between them and many bakery products. Besides, it is also correlated with the Organic Raw Multigreen Kombucha, which is in the drinks departments, and two other products from the babies department. However, and considering also results in previous exercises, we have to consider the possibility of this being a result of a general like for bananas, which are frequently bought and hence can be correlated with some products that are also bought with relative frequency. To determine with more accuracy the meaning of these rules we should develop further analysis on this topic such as frequency with which products that are now related to bananas are bought, if they are correlated to other products, etc.

However, we must take into consideration that the confidence in all these relations is not very high (always <0.7), which can make us have a stronger belief in the theory presented above, since the confidence is not so strong to state without a doubt that the products are bought together systematically.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>